In [ ]:
#Goal: Recreate N2 using custom ECFs
import numpy as np
import awkward as ak
from coffea import processor
import json
import hist
from coffea.nanoevents import NanoEventsFactory, BaseSchema, NanoAODSchema
import matplotlib.pyplot as plt
from lpcjobqueue import LPCCondorCluster
from distributed import Client
from coffea.lookup_tools.lookup_base import lookup_base
from coffea import util
import importlib.resources
import gzip
import pickle
import os
import fastjet
from custom_n2 import e2
from e23_numba import e23_loop, e23_numba, e2_loop, e2_numba
import numba

In [ ]:
cluster = LPCCondorCluster(ship_env=True, 
                           transfer_input_files=['custom_ecf.py', 'ecf_numba.py'], 
                           #log_directory='/uscmst1b_scratch/lpc1/3DayLifetime/cjmoore/mylog',
                           memory='1258291200'
                          )        
LPCCondorCluster()
cluster.adapt(minimum=0, maximum=100)
client = Client(cluster)

In [ ]:
#print(LPCCondorCluster.__doc__)

In [ ]:
client

In [ ]:
#json file with signal and background samples
with open("jsons/filelist_sans_bad_files.json") as fin:
    filesets = json.load(fin)

In [ ]:
#coffea processor
from custom_ecf import e2, e23, distance
from ecf_numba import e23_loop, e23_numba, e2_loop, e2_numba

class MyProcessor(processor.ProcessorABC):
    
    def __init__(self):
        pass
    
    def process(self, events):
        dataset = events.metadata['dataset']
       
        fatjet = events.FatJet
        pfcands = events.PFCands
        fatjetpfcands = events.FatJetPFCands
        
        n2 = (
            hist.Hist.new
            .Reg(60, -1.4, 0.25, name='n2b1', label='N2B1')
            .Double()
        )
        
        n2.fill(n2b1=ak.flatten(fatjet.n2b1))
        
        custom_n2_mix = (
            hist.Hist.new
            .Reg(60, -1.4, 0.25, name='my_n2', label='my_n2_mix')
            .Double()
        )
        
        custom_n2_numba = (
            hist.Hist.new
            .Reg(60, -1.4, 0.25, name='my_n2', label='my_n2_numba')
            .Double()
        )
        
        custom_n2_awkward = (
            hist.Hist.new
            .Reg(60, -1.4, 0.25, name='my_n2', label='my_n2_awkward')
            .Double()
        )
        
        s_mass = (
            hist.Hist.new
            .Reg(40, 0, 200., name='softdrop_mass', label='FatJet_msoftdrop')
            .Double()
        )
        
        #fatjet['n2ddt'] = fatjet.n2b1 - n2ddt_shift(fatjet, year='2017')
        fatjet['bespoke_n2b1_mix'] = e23_numba(fatjetpfcands, pfcands, fatjet)/(e2(fatjetpfcands, pfcands, fatjet)**2)
        fatjet['bespoke_n2b1_numba'] = e23_numba(fatjetpfcands, pfcands, fatjet)/(e2_numba(fatjetpfcands, pfcands, fatjet)**2)
        fatjet['bespoke_n2b1_awkward'] = e23(fatjetpfcands, pfcands, fatjet)/(e2(fatjetpfcands, pfcands, fatjet)**2)
        #fatjet['e2'] = e2(fatjetpfcands, pfcands, fatjet)
        #fatjet['e23'] = e23_numba(fatjetpfcands, pfcands, fatjet)
        
        custom_n2_mix.fill(my_n2=ak.flatten(fatjet.bespoke_n2b1_mix))
        custom_n2_numba.fill(my_n2=ak.flatten(fatjet.bespoke_n2b1_numba))
        custom_n2_awkward.fill(my_n2=ak.flatten(fatjet.bespoke_n2b1_awkward))
        s_mass.fill(softdrop_mass=ak.flatten(fatjet.msoftdrop))
        
        return {
            dataset: {
                "entries": len(events),
                "n2b1": n2,
                "custom_n2_mix": custom_n2_mix,
                "softdrop_mass": s_mass,
                "custom_n2_numba": custom_n2_numba,
                "custom_n2_awkward": custom_n2_awkward,
            }
        }
    def postprocess(self, accumulator):
        pass

In [ ]:
#Run processor on LPC condor
processor_instance=MyProcessor()
dask_run = processor.Runner(
    executor = processor.DaskExecutor(client=client),
    schema=NanoAODSchema,
    chunksize = 1000,
    maxchunks=500,
)

out = dask_run(
    filesets,
    "Events",
    processor_instance=MyProcessor()
)
out

In [ ]:
import warnings
warnings.filterwarnings("ignore", "Found duplicate branch")
processor_instance=MyProcessor()
run = processor.Runner(
    executor = processor.FuturesExecutor(compression=None, workers=20),
    schema=NanoAODSchema,
    chunksize=500,
    #maxchunks=500,
)

out = run(
    filesets,
    "Events",
    processor_instance=MyProcessor()
)
out

In [ ]:
f = open("n2_.pkl","wb")

# write the python object (dict) to pickle file
pickle.dump(out,f)

# close file
f.close()

In [ ]:
file = open("n2.pkl", "rb")
out = pickle.load(file)
#out

In [ ]:
out['HJ']

In [ ]:
qcd_n2b1 = (out['QCD_Pt_470to600_TuneCP5_13TeV_pythia8']['n2b1']+
        out['QCD_Pt_600to800_TuneCP5_13TeV_pythia8']['n2b1']+
        out['QCD_Pt_800to1000_TuneCP5_13TeV_pythia8']['n2b1']+
        out['QCD_Pt_1000to1400_TuneCP5_13TeV_pythia8']['n2b1']+
        out['QCD_Pt_1400to1800_TuneCP5_13TeV_pythia8']['n2b1']+
        out['QCD_Pt_1800to2400_TuneCP5_13TeV_pythia8']['n2b1']+
        out['QCD_Pt_2400to3200_TuneCP5_13TeV_pythia8']['n2b1']+
        out['QCD_Pt_3200toInf_TuneCP5_13TeV_pythia8']['n2b1'])

qcd_custom_n2 = (out['QCD_Pt_470to600_TuneCP5_13TeV_pythia8']['custom_n2']+
        out['QCD_Pt_600to800_TuneCP5_13TeV_pythia8']['custom_n2']+
        out['QCD_Pt_800to1000_TuneCP5_13TeV_pythia8']['custom_n2']+
        out['QCD_Pt_1000to1400_TuneCP5_13TeV_pythia8']['custom_n2']+
        out['QCD_Pt_1400to1800_TuneCP5_13TeV_pythia8']['custom_n2']+
        out['QCD_Pt_1800to2400_TuneCP5_13TeV_pythia8']['custom_n2']+
        out['QCD_Pt_2400to3200_TuneCP5_13TeV_pythia8']['custom_n2']+
        out['QCD_Pt_3200toInf_TuneCP5_13TeV_pythia8']['custom_n2'])

In [ ]:
#plotting
fig, ax = plt.subplots()
#out['HJ']['n2b1'].plot1d(ax=ax)
#out['HJ']['custom_n2'].plot1d(ax=ax)
qcd_n2b1.plot1d(ax=ax)
qcd_custom_n2.plot1d(ax=ax)
ax.legend(['QCD n2',
           'QCD custom n2'

          ], 
           title='N2 Source'
         )
plt.title('QCD N2 Comparison')
ax.set_xlim([-0.025,0.275])